In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
island_crops = pd.read_csv('data/compatibility_data/FAOSTAT_crop_data_islands.csv')
country_crops = pd.read_csv('data/compatibility_data/FAOSTAT_crop_data_countries.csv')

In [11]:
island_crops = island_crops[['Area','Item Code','Item','Value']]
island_crops = island_crops[island_crops['Value'] > 100]
island_crops['Item Code'] = island_crops['Item Code'].astype('int32')
island_crops

,Area,Item Code,Item,Value
0,Kiribati,486,Bananas,8339
1,Kiribati,249,Coconuts,174135
2,Kiribati,603,"Fruit, tropical fresh nes",1677
3,Kiribati,234,Nuts nes,285
4,Kiribati,149,Roots and tubers nes,9896
5,Kiribati,136,Taro (cocoyam),1925
6,Kiribati,463,"Vegetables, fresh nes",5767
8,Maldives,486,Bananas,334
12,Maldives,249,Coconuts,225
13,Maldives,619,"Fruit, fresh nes",2057


In [12]:
country_crops = country_crops[['Area','Item Code','Item','Value']]
country_crops = country_crops[country_crops['Value'] > 100]
country_crops['Item Code'] = country_crops['Item Code'].astype('int32')
country_crops

,Area,Item Code,Item,Value
0,Australia,221,"Almonds, with shell",69880.0
1,Australia,711,"Anise, badian, fennel, coriander",1264.0
2,Australia,515,Apples,268355.0
3,Australia,526,Apricots,8801.0
4,Australia,367,Asparagus,7233.0
...,...,...,...,...
1116,United States of America,463,"Vegetables, fresh nes",807349.0
1117,United States of America,420,"Vegetables, leguminous nes",43986.0
1118,United States of America,222,"Walnuts, with shell",613260.0
1119,United States of America,567,Watermelons,1771051.0


In [5]:
country_crops[country_crops['Area'] == 'United States of America'] = 'United States'
country_crops[country_crops['Area'] == 'Russian Federation'] = 'Russia'
country_crops[country_crops['Area'] == 'China, Hong Kong SAR'] = 'Hong Kong'
country_crops[country_crops['Area'] == 'China, Taiwan Province of China'] = 'Taiwan'
country_crops[country_crops['Area'] == 'China, mainland'] = 'China'

In [17]:
island_crops.to_csv('data/compatibility_data/island_crops.csv',index=False)
country_crops.to_csv('data/compatibility_data/country_crops.csv',index=False)

In [48]:
def compatibility(c1,c2):
    
    def _common_crops(c1,c2):
        island_crops = pd.read_csv('data/compatibility_data/island_crops.csv')
        country_crops = pd.read_csv('data/compatibility_data/country_crops.csv')
        df = pd.merge(island_crops[island_crops['Area']==c1],\
                      country_crops[country_crops['Area']==c2],\
                      how='inner', on='Item Code')
        return df.shape[0] / 173
    
    def _latitude_diff(c1,c2):
        latitude = pd.read_csv('data/compatibility_data/latitude.csv')
        return abs(latitude.loc[c1]['Latitude'] - latitude.loc[c2]['Latitude']) / 180
    
    def _religion_sim(c1,c2):
        island_religions = {'Marshall Islands':'Christian',
                            'Tuvalu':'Christian',
                            'Maldives':'Muslim',
                            'Kiribati':'Christian'}
        country_religions = pd.read_csv('data/compatibility_data/religion_pct_by_country.csv')
        
        religion = island_religions[c1]
        
        return country_religions.loc[c2][religion] / 100
    
    return np.average([_common_crops(c1,c2), 1 - _latitude_diff(c1,c2), _religion_sim(c1,c2)])

In [53]:
compatibility('Tuvalu','United States')

0.5124824074074074